In [1]:
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.units import DistanceUnit

driver = PySCFDriver(
    atom="H 0 0 0; H 0 0 0.735",
    basis="sto3g",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)
problem = driver.run()
hamiltonian = problem.hamiltonian


In [5]:
import numpy as np 
from qiskit_nature.second_q.operators import PolynomialTensor
from qiskit_nature.second_q.properties import ElectronicDipoleMoment

dipole : ElectronicDipoleMoment
dipole = problem.properties.electronic_dipole_moment

if dipole is not None:
   
    nuclear_dip = dipole.nuclear_dipole_moment

    
    dipole.x_dipole.alpha += PolynomialTensor({"": nuclear_dip[0]})
    dipole.y_dipole.alpha += PolynomialTensor({"": nuclear_dip[1]})
    dipole.z_dipole.alpha += PolynomialTensor({"": nuclear_dip[2]})

    Gamma = 0.25
    E0 = 0.01  
    t = 3


    
    dipole_term = ((E0 / np.pi) * Gamma/(Gamma**2 + t**2)) * (
        dipole.x_dipole.alpha + dipole.y_dipole.alpha + dipole.z_dipole.alpha
    )

    
    hamiltonian.electronic_integrals.alpha += dipole_term

    print("Đã thêm moment lưỡng cực vào Hamiltonian .")
else:
    print("Moment lưỡng cực không tồn tại trong problem.")


Đã thêm moment lưỡng cực vào Hamiltonian .


In [6]:
coefficients = hamiltonian.electronic_integrals
print(coefficients.alpha)


Polynomial Tensor
 "+-":
array([[-1.25615613e+00, -2.44418124e-04],
       [-2.44418124e-04, -4.71713063e-01]])
 "++--":
array([6.75710155e-01, 1.39486891e-16, 1.80931200e-01, 6.64581730e-01,
       2.57172666e-16, 6.98573723e-01])
 "":
array(0.00073178)


In [7]:
second_q_op = hamiltonian.second_q_op()
print(second_q_op)

Fermionic Operator
number spin orbitals=4, number terms=41
  0.0007317783767373089
+ -1.2561561284088605 * ( +_0 -_0 )
+ -0.0002444181239001983 * ( +_0 -_1 )
+ -0.00024441812390024536 * ( +_1 -_0 )
+ -0.47171306268675267 * ( +_1 -_1 )
+ -1.2561561284088605 * ( +_2 -_2 )
+ -0.0002444181239001983 * ( +_2 -_3 )
+ -0.00024441812390024536 * ( +_3 -_2 )
+ -0.47171306268675267 * ( +_3 -_3 )
+ 0.33785507740175813 * ( +_0 +_0 -_0 -_0 )
+ 0.33229086512764816 * ( +_0 +_1 -_1 -_0 )
+ 0.33785507740175813 * ( +_0 +_2 -_2 -_0 )
+ 0.33229086512764816 * ( +_0 +_3 -_3 -_0 )
+ 0.0904655998921157 * ( +_0 +_0 -_1 -_1 )
+ 0.0904655998921157 * ( +_0 +_1 -_0 -_1 )
+ 0.0904655998921157 * ( +_0 +_2 -_3 -_1 )
+ 0.0904655998921157 * ( +_0 +_3 -_2 -_1 )
+ 0.0904655998921157 * ( +_1 +_0 -_1 -_0 )
+ 0.0904655998921157 * ( +_1 +_1 -_0 -_0 )
+ 0.0904655998921157 * ( +_1 +_2 -_3 -_0 )
+ 0.0904655998921157 * ( +_1 +_3 -_2 -_0 )
+ 0.33229086512764816 * ( +_1 +_0 -_0 -_1 )
+ 0.34928686136600906 * ( +_1 +_1 -_1 -_1 )
+ 0.3

In [ ]:
from qiskit_nature.second_q.mappers import JordanWignerMapper
#from qiskit_nature.second_q.mappers import ParityMapper

mapper = JordanWignerMapper()
qubit_p_op = mapper.map(second_q_op)

In [9]:
print(qubit_p_op)

SparsePauliOp(['IIII', 'IIIZ', 'IIYY', 'IIXX', 'IIZI', 'IZII', 'YYII', 'XXII', 'ZIII', 'IIZZ', 'IZIZ', 'ZIIZ', 'YYYY', 'XXYY', 'YYXX', 'XXXX', 'IZZI', 'ZIZI', 'ZZII'],
              coeffs=[-8.09450313e-01+0.j,  1.72092460e-01+0.j, -1.22209062e-04+0.j,
 -1.22209062e-04+0.j, -2.25844965e-01+0.j,  1.72092460e-01+0.j,
 -1.22209062e-04+0.j, -1.22209062e-04+0.j, -2.25844965e-01+0.j,
  1.20912633e-01+0.j,  1.68927539e-01+0.j,  1.66145433e-01+0.j,
  4.52327999e-02+0.j,  4.52327999e-02+0.j,  4.52327999e-02+0.j,
  4.52327999e-02+0.j,  1.66145433e-01+0.j,  1.74643431e-01+0.j,
  1.20912633e-01+0.j])


In [10]:
from qiskit.quantum_info import SparsePauliOp
Hopt = SparsePauliOp(['IIII', 'IIIZ', 'IIYY', 'IIXX', 'IIZI', 'IZII', 'YYII', 'XXII', 'ZIII', 'IIZZ', 'IZIZ', 'ZIIZ', 'YYYY', 'XXYY', 'YYXX', 'XXXX', 'IZZI', 'ZIZI', 'ZZII'],
              coeffs=[-8.09450313e-01+0.j,  1.72092460e-01+0.j, -1.22209062e-04+0.j,
 -1.22209062e-04+0.j, -2.25844965e-01+0.j,  1.72092460e-01+0.j,
 -1.22209062e-04+0.j, -1.22209062e-04+0.j, -2.25844965e-01+0.j,
  1.20912633e-01+0.j,  1.68927539e-01+0.j,  1.66145433e-01+0.j,
  4.52327999e-02+0.j,  4.52327999e-02+0.j,  4.52327999e-02+0.j,
  4.52327999e-02+0.j,  1.66145433e-01+0.j,  1.74643431e-01+0.j,
  1.20912633e-01+0.j])

In [11]:
import numpy as np
import qiskit
from scipy.linalg import expm
from qiskit.circuit.library import UnitaryGate
def time_dependent_qc(num_qubits: int, Hopt, t):
    """Create U circuit from Hopt and time t
    
    Args:
        - num_qubits (int): Number of qubits
        - Hopt: Hamiltonian (SparsePauliOp)
        - t (float): Time
        
    Returns:
        - QuantumCircuit: The quantum circuit with time evolution
    """
    # Create quantum circuit
    qc = qiskit.QuantumCircuit(num_qubits, num_qubits)
    
    # Ensure Hopt is Hermitian
    if not np.allclose(Hopt.to_matrix(), np.conj(Hopt.to_matrix()).T):
        raise ValueError("The Hamiltonian is not Hermitian.")
    
    # Calculate the unitary matrix U = exp(-i * H * t)
    U = expm(-1j * t * Hopt.to_matrix())

    # Check if U is unitary
    if not np.allclose(U @ U.conj().T, np.eye(U.shape[0])):
        raise ValueError("The resulting matrix U is not unitary.")

    # Create a UnitaryGate from the unitary matrix U
    unitary_gate = UnitaryGate(U)

    # Append the unitary gate to the quantum circuit
    qc.append(unitary_gate, range(qc.num_qubits))

    return qc



In [12]:
# Tạo mạch lượng tử
num_qubits = 4
t = 6.0  # Time evolution duration
qc = time_dependent_qc(num_qubits, Hopt, t)

# Convert the matrix U to the required format
U = expm(-1j * t * Hopt.to_matrix())


In [13]:

from qoop.compilation.qsp import QuantumStatePreparation
compiler = QuantumStatePreparation.prepare(U)


In [14]:
compiler.u.draw()

┌──────────────┐┌──────────────┐ ┌──────────────┐               ┌───┐
q_0: ┤ Rz(theta[0]) ├┤ Rx(theta[4]) ├─┤ Rz(theta[8]) ├──■────────────┤ X ├
     ├──────────────┤├──────────────┤ ├──────────────┤┌─┴─┐          └─┬─┘
q_1: ┤ Rz(theta[1]) ├┤ Rx(theta[5]) ├─┤ Rz(theta[9]) ├┤ X ├──■─────────┼──
     ├──────────────┤├──────────────┤┌┴──────────────┤└───┘┌─┴─┐       │  
q_2: ┤ Rz(theta[2]) ├┤ Rx(theta[6]) ├┤ Rz(theta[10]) ├─────┤ X ├──■────┼──
     ├──────────────┤├──────────────┤├───────────────┤     └───┘┌─┴─┐  │  
q_3: ┤ Rz(theta[3]) ├┤ Rx(theta[7]) ├┤ Rz(theta[11]) ├──────────┤ X ├──■──
     └──────────────┘└──────────────┘└───────────────┘          └───┘

In [15]:
compiler.thetas

array([ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
       -2.71885275e-06, -2.71885275e-06, -2.71885275e-06, -2.71885275e-06,
        9.99999995e-01,  9.99999993e-01,  1.00000000e+00,  9.99999993e-01])